In [ ]:
import pandas as pd
import wandb
import json

# Functions

In [ ]:
attribute_name = "Test Node"
attributes = ["- Third Disp.", "- Second Disp.", "- Disp.", "- Acc.", "- Group 3"]

mapping = {
    "- Third Disp.": "DP_RACE",
    "- Second Disp.": "DP_MAR",
    "- Disp.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
def create_value_plot(
    df: pd.DataFrame,
    y_label: str,
    title: str,
    attribute: str,
    font_size_labels: int = 22,
    font_size_title: int = 22,
    font_size_ticks: int = 22,
    file_name: str = None,
    save: bool = False
):
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Define a color-blind-friendly color
    marker_face_color = '#56B4E9'  # Blue from ColorBrewer Set2
    marker_edge_color = 'black'    # Black edges for visibility

    for val in df['Value'].unique():
        y_vals = df[df['Value'] == val][attribute]
        x_vals = [int(float(val))] * len(y_vals)
        plt.scatter(
            x_vals,
            y_vals,
            facecolors=marker_face_color,
            edgecolors=marker_edge_color,
            marker='o',
            s=200,
            linewidths=1.2
        )

    plt.xlabel('Sensitive Group Value', fontsize=font_size_labels)
    plt.ylabel(y_label, fontsize=font_size_labels)
    plt.title(title, fontsize=font_size_title)

    max_val = int(float(max(df['Value'])))
    labels = list(range(1, max_val + 1))
    plt.xticks(ticks=labels, labels=labels, fontsize=font_size_ticks)
    plt.yticks(fontsize=font_size_ticks)
    plt.grid(True)

    if save:
        plt.savefig(f"./images/{file_name}.pdf", bbox_inches='tight', dpi=150)
    else:
        plt.tight_layout()
        plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter


def compute_differences(df1, df2):
    # Set the 'State' column as index if not already
    df1 = df1.set_index('dataset')
    df2 = df2.set_index('dataset')

    # Compute differences
    diff_df = pd.DataFrame({
        'DP_RACE': df1['DP_RACE'] - df2['DP_RACE'],
        'DP_SEX': df1['DP_SEX'] - df2['DP_SEX'],
    })
    # Add dataset name metadata as columns
    # df1 = df1.reset_index()
    # diff_df["dataset"] = df1["dataset"]
    # Optional: Reset index if you want 'State' as a column
    # diff_df = diff_df.reset_index()
    diff_df = diff_df.reset_index()
    diff_df.head()
    return diff_df


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def plot_comparison_fairness(df: pd.DataFrame, title: str):
    custom_palette = {"DP_SEX": "#1E88E5", "DP_RACE": "#D81B60"}

    # 1) Count maximum unfairness and represent client unfairness per attribute
    df['max_unfairness'] = df[['DP_SEX', 'DP_RACE']].max(axis=1)
    unfairness_counts = df[['DP_SEX', 'DP_RACE']].apply(lambda row: row[row == row.max()].index.tolist(), axis=1)

    # Flatten the list of lists and count occurrences
    attribute_unfairness_counts = Counter([item for sublist in unfairness_counts for item in sublist])

    plt.figure(figsize=(8, 6))
    sns.barplot(
        x=list(attribute_unfairness_counts.keys()),
        y=list(attribute_unfairness_counts.values()),
        palette=custom_palette
    )
    plt.title(f'Number of Clients with Maximum Unfairness Towards Each Attribute - {title}')
    plt.xlabel('Sensitive Attribute')
    plt.ylabel('Number of Clients')
    plt.show()

    # 2) Visualize clients unfair toward different groups
    df_reset = df.reset_index()  # Make the index a regular column
    df_melted = df_reset.melt(
        id_vars='index',
        value_vars=['DP_SEX', 'DP_RACE'],
        var_name='Sensitive Attribute',
        value_name='Unfairness Score'
    )

    ticks = list(range(0, min(50, len(df))))  # Adjusted for robustness
    plt.figure(figsize=(12, 6))
    sns.barplot(
        x='index',
        y='Unfairness Score',
        hue='Sensitive Attribute',
        data=df_melted,
        palette=custom_palette
    )
    plt.title(f'Unfairness Score of Each Client Towards Different Sensitive Attributes - {title}')
    plt.xlabel('Client Index')
    plt.ylabel('Unfairness Score')
    plt.legend(title='Sensitive Attribute')
    plt.xticks(ticks, rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # 3) Cumulative Distribution Function of unfairness
    plt.figure(figsize=(10, 6))
    for col in ['DP_SEX', 'DP_RACE']:
        sns.kdeplot(df[col], cumulative=True, label=col, color=custom_palette[col])

    plt.title(f'Cumulative Distribution Function of Unfairness - {title}')
    plt.xlabel('Unfairness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend(title='Sensitive Attribute')
    plt.grid(True)
    plt.show()


import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def bar_plot_differences(df, labels, title,
                                        font_size_title=25, font_size_ticks=22, font_size_labels=24, y_axis="",
                                        save: bool = False, fig_name: str = "", legend_name: str = "bar_plot_differences_legend"):
    """
    Creates a grouped bar plot of unfairness scores for different sensitive attributes,
    with the legend saved as a separate image.

    Args:
        df (pd.DataFrame): DataFrame with 'State' as index and 'DP_SEX', 'DP_RACE' columns.
        labels (list): List of state labels for the x-axis.
        title (str): Title of the plot.
        font_size_title (int, optional): Font size for the title. Defaults to 25.
        font_size_ticks (int, optional): Font size for the axis ticks. Defaults to 22.
        font_size_labels (int, optional): Font size for the axis labels. Defaults to 24.
        y_axis (str, optional): Label for the y-axis. Defaults to "".
        save (bool, optional): Whether to save the plot. Defaults to False.
        fig_name (str, optional): Filename for saving the plot. Defaults to "".
        legend_name (str, optional): Filename for saving the legend. Defaults to "bar_plot_differences_legend".
    """
    df_reset = df.reset_index().rename(columns={'index': 'State'})
    df_melted = df_reset.melt(id_vars='State',
                              value_vars=['DP_SEX', 'DP_RACE'],
                              var_name='Sensitive Attribute',
                              value_name='Unfairness Score')

    ticks = list(range(len(labels)))
    fig, ax = plt.subplots(figsize=(16, 6))  # Create the main figure and axes

    # Define custom colors
    custom_palette = {"DP_SEX": "#1E88E5", "DP_RACE": "#D81B60"}

    sns.barplot(
        x='State', y='Unfairness Score',
        hue='Sensitive Attribute', data=df_melted,
        palette=custom_palette, ax=ax  # Pass the axes to seaborn
    )

    # Title and labels
    ax.set_title(title, fontsize=font_size_title, pad=20)
    ax.set_xlabel('State', fontsize=font_size_labels, labelpad=15)
    ax.set_ylabel(y_axis, fontsize=font_size_labels, labelpad=15)

    # Ticks
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=90, ha='right', fontsize=font_size_ticks)
    ax.tick_params(axis='y', labelsize=font_size_ticks)

    # Get the legend object
    legend = ax.get_legend()
    if legend:
        # Remove the legend from the main plot
        legend.remove()

        if save:
            # Create a separate figure for the legend
            fig_legend = plt.figure(figsize=(6, 1))  # Adjust size as needed
            ax_legend = fig_legend.add_subplot(111)
            # Re-draw the legend on the separate figure
            handles, labels_legend = ax.get_legend_handles_labels()
            ax_legend.legend(handles, labels_legend,
                               loc='center',
                               fontsize=font_size_ticks,
                               ncol=2, frameon=False)
            ax_legend.axis('off')
            fig_legend.tight_layout()
            plt.savefig(f"./images/{legend_name}.pdf", bbox_inches='tight', dpi=150)
            plt.close(fig_legend)

        # Save the main plot
        if save:
            plt.savefig(f"./images/{fig_name}.pdf", bbox_inches='tight', dpi=150)
        else:
            plt.show()
        
    else:
        # Save the main plot even if there's no legend
        if save:
            plt.savefig(f"./images/{fig_name}.pdf", bbox_inches='tight', dpi=150)
        else:
            plt.show()


import matplotlib.pyplot as plt

# def scatter_fairness_plot(
#     df1: pd.DataFrame,
#     client_column: str = "Partition ID",
#     fairness_column_Y: str = "RAC1P_DP",
#     fairness_column_X: str = "RAC1P_DP",
#     title: str = "Fairness Before/After Comparison",
#     figsize: tuple = (6, 6),
#     ylabel: str = "Fairness Value Before",
#     xlabel: str = "Fairness Value After",
#     title_font_size: int = 25,
#     label_font_size: int = 25,
#     ticks_font_size: int = 20,
# ) -> plt.Figure:
#     """
#     Plot a scatter comparison of two fairness metrics from the same DataFrame.

#     Parameters
#     ----------
#     df1 : pd.DataFrame
#         DataFrame containing client IDs and two fairness metrics.

#     client_column : str, default="Partition ID"
#         Name of the column containing the client IDs.

#     fairness_column_Y : str, default="RAC1P_DP"
#         Column for y-axis values.

#     fairness_column_X : str, default="RAC1P_DP"
#         Column for x-axis values.

#     Returns
#     -------
#     matplotlib.figure.Figure
#         The matplotlib figure object containing the plot.
#     """
#     assert df1[client_column].is_unique, "The client ID column must be unique."

#     fairness1 = df1[fairness_column_X]
#     fairness2 = df1[fairness_column_Y]

#     min_val = min(fairness1.min(), fairness2.min())
#     max_val = max(fairness1.max(), fairness2.max())
#     padding = 0.05 * (max_val - min_val)

#     fig, ax = plt.subplots(figsize=figsize)

#     # Use color-blind friendly palette
#     marker_face_color = '#56B4E9'  # Green tone
#     marker_edge_color = 'black'

#     ax.scatter(
#         fairness1,
#         fairness2,
#         facecolors=marker_face_color,
#         edgecolors=marker_edge_color,
#         marker='o',
#         s=200,
#         linewidths=1.2,
#         alpha=0.8
#     )

#     ax.plot(
#         [min_val - padding, max_val + padding],
#         [min_val - padding, max_val + padding],
#         linestyle="dotted",
#         color="gray",
#         linewidth=2
#     )

#     ax.tick_params(axis="both", which="major", labelsize=ticks_font_size)
#     ax.set_xlim(min_val - padding, max_val + padding)
#     ax.set_ylim(min_val - padding, max_val + padding)
#     ax.set_xlabel(xlabel, fontsize=label_font_size)
#     ax.set_ylabel(ylabel, fontsize=label_font_size)
#     ax.set_title(title, fontsize=title_font_size)
#     ax.grid(True)

#     return fig

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def scatter_fairness_plot(
    df1: pd.DataFrame,
    client_column: str = "Partition ID",
    fairness_column_Y: str = "RAC1P_DP",
    fairness_column_X: str = "RAC1P_DP",
    title: str = "Fairness Comparison",
    figsize: tuple = (6, 6),
    ylabel: str = "Fairness Metric Y",
    xlabel: str = "Fairness Metric X",
    title_font_size: int = 25,
    label_font_size: int = 25,
    ticks_font_size: int = 20,
    unfairness_distribution: dict = None,
    legend_filename: str = "fairness_plot_legend.png",
) -> plt.Figure:
    """
    Plot a scatter comparison of two fairness metrics from the same DataFrame,
    coloring points based on state lists, with the legend saved separately.
    """
    assert df1[client_column].is_unique, "The client ID column must be unique."

    fairness_x = df1[fairness_column_X]
    fairness_y = df1[fairness_column_Y]
    clients = df1[client_column]

    min_val = min(fairness_x.min(), fairness_y.min())
    max_val = max(fairness_x.max(), fairness_y.max())
    padding = 0.05 * (max_val - min_val)

    fig, ax = plt.subplots(figsize=figsize)


    # Define colors for the two groups
    race_color = '#D81B60'  # Vivid orange
    sex_color = '#1E88E5'   # Blue

    # Lists to store data for each group
    race_x = []
    race_y = []
    sex_x = []
    sex_y = []

    if unfairness_distribution:
        race_states = unfairness_distribution.get("race_state", [])
        sex_states = unfairness_distribution.get("sex_states", [])

        for i, client in enumerate(clients):
            if client in race_states:
                race_x.append(fairness_x.iloc[i])
                race_y.append(fairness_y.iloc[i])
            elif client in sex_states:
                sex_x.append(fairness_x.iloc[i])
                sex_y.append(fairness_y.iloc[i])
            else:
                # We are not including the 'other' states
                pass
    else:
        # If no unfairness_distribution is provided, plot all points with a default color
        ax.scatter(
            fairness_x,
            fairness_y,
            facecolors='#56B4E9',
            edgecolors='black',
            marker='o',
            s=200,
            linewidths=1.2,
            alpha=0.8,
        )
        ax.plot(
            [min_val - padding, max_val + padding],
            [min_val - padding, max_val + padding],
            linestyle="dotted",
            color="gray",
            linewidth=2
        )
        ax.tick_params(axis="both", which="major", labelsize=ticks_font_size)
        ax.set_xlim(min_val - padding, max_val + padding)
        ax.set_ylim(min_val - padding, max_val + padding)
        ax.set_xlabel(xlabel, fontsize=label_font_size)
        ax.set_ylabel(ylabel, fontsize=label_font_size)
        ax.set_title(title, fontsize=title_font_size)
        ax.grid(True)
        fig.tight_layout()
        return fig

    # Scatter plot for race-related states
    ax.scatter(
        race_x,
        race_y,
        facecolors=race_color,
        edgecolors='black',
        marker='o',
        s=200,
        linewidths=1.2,
        alpha=0.8,
        label='Race-Related States'
    )

    # Scatter plot for sex-related states
    ax.scatter(
        sex_x,
        sex_y,
        facecolors=sex_color,
        edgecolors='black',
        marker='o',
        s=200,
        linewidths=1.2,
        alpha=0.8,
        label='Sex-Related States'
    )

    ax.plot(
        [min_val - padding, max_val + padding],
        [min_val - padding, max_val + padding],
        linestyle="dotted",
        color="gray",
        linewidth=2
    )

    ax.tick_params(axis="both", which="major", labelsize=ticks_font_size)
    ax.set_xlim(min_val - padding, max_val + padding)
    ax.set_ylim(min_val - padding, max_val + padding)
    ax.set_xlabel(xlabel, fontsize=label_font_size)
    ax.set_ylabel(ylabel, fontsize=label_font_size)
    ax.set_title(title, fontsize=title_font_size)
    ax.grid(True)

    # Create a separate figure for the legend
    fig_legend = plt.figure(figsize=(6, 1))
    ax_legend = fig_legend.add_subplot(111)

    # Create custom patches for the legend
    race_patch = mpatches.Patch(facecolor=race_color, edgecolor='black', label='Race-Related States')
    sex_patch = mpatches.Patch(facecolor=sex_color, edgecolor='black', label='Sex-Related States')

    # Add the legend to the separate axes
    ax_legend.legend(handles=[race_patch, sex_patch], fontsize=24, loc='center', frameon=False, ncol=2)
    ax_legend.axis('off')  # Turn off the axes for the legend

    fig_legend.tight_layout()
    fig_legend.savefig(legend_filename)

    fig.tight_layout()

    return fig


In [ ]:
# def local_client_fairness_plot(
#     df1: pd.DataFrame,
#     df2: pd.DataFrame,
#     client_column: str = "Partition ID",
#     fairness_column: str = "RAC1P_DP",
#     title: str = "Fairness Before/After Comparison",
#     figsize: tuple = (6, 6),
#     ylabel: str = "Fairness Value Before",
#     xlabel: str = "Fairness Value After",
#     title_font_size: int = 25,
#     label_font_size: int = 25,
#     ticks_font_size: int = 20,
#     unfairness_distribution: dict = None,
    
# ) -> plt.Figure:
#     """
#     Plot a scatter comparison of fairness values from two dataframes.
#     """
#     assert df1[client_column].is_unique, "The client ID column must be unique."

#     merged = pd.merge(
#         df1[[client_column, fairness_column]].rename(columns={fairness_column: "fairness1"}),
#         df2[[client_column, fairness_column]].rename(columns={fairness_column: "fairness2"}),
#         on=client_column,
#     )

#     fairness1 = merged["fairness1"]
#     fairness2 = merged["fairness2"]
#     min_val = min(fairness1.min(), fairness2.min())
#     max_val = max(fairness1.max(), fairness2.max())
#     padding = 0.05 * (max_val - min_val)

#     fig, ax = plt.subplots(figsize=figsize)
    

#     # Color-blind-friendly style
#     marker_face_color = '#56B4E9'  # Orange from color-blind safe palette
#     marker_edge_color = 'black'


#     ax.scatter(
#         fairness2,
#         fairness1,
#         facecolors=marker_face_color,
#         edgecolors=marker_edge_color,
#         marker='o',
#         s=200,
#         linewidths=1.2,
#         alpha=0.8
#     )

#     ax.plot(
#         [min_val - padding, max_val + padding],
#         [min_val - padding, max_val + padding],
#         linestyle="dotted",
#         color="gray",
#         linewidth=2
#     )
    
#     ax.tick_params(axis="both", which="major", labelsize=ticks_font_size)
#     ax.set_xlim(min_val - padding, max_val + padding)
#     ax.set_ylim(min_val - padding, max_val + padding)
#     ax.set_xlabel(xlabel, fontsize=label_font_size)
#     ax.set_ylabel(ylabel, fontsize=label_font_size)
#     ax.set_title(title, fontsize=title_font_size)
#     ax.grid(True)

#     fig.tight_layout()

#     return fig

import matplotlib.patches as mpatches
import pandas as pd
import matplotlib.pyplot as plt

def local_client_fairness_plot(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    client_column: str = "Partition ID",
    fairness_column: str = "RAC1P_DP",
    title: str = "Fairness Before/After Comparison",
    figsize: tuple = (6, 6),
    ylabel: str = "Fairness Value Before",
    xlabel: str = "Fairness Value After",
    title_font_size: int = 25,
    label_font_size: int = 25,
    ticks_font_size: int = 20,
    unfairness_distribution: dict = None,

) -> plt.Figure:
    """
    Plot a scatter comparison of fairness values from two dataframes,
    coloring points based on state lists.
    """
    assert df1[client_column].is_unique, "The client ID column must be unique."

    merged = pd.merge(
        df1[[client_column, fairness_column]].rename(columns={fairness_column: "fairness1"}),
        df2[[client_column, fairness_column]].rename(columns={fairness_column: "fairness2"}),
        on=client_column,
    )
    print(merged.shape)

    fairness1 = merged["fairness1"]
    fairness2 = merged["fairness2"]
    clients = merged[client_column]
    min_val = min(fairness1.min(), fairness2.min())
    max_val = max(fairness1.max(), fairness2.max())
    padding = 0.05 * (max_val - min_val)

    fig, ax = plt.subplots(figsize=figsize)
    # Define colors for the two groups
    if unfairness_distribution:
    
        race_color = '#D81B60'  # Vivid orange
        sex_color = '#1E88E5'   # Blue
    else:
        race_color = '#56B4E9'  # Vivid orange
        sex_color = '#56B4E9'   # Blue
    # Lists to store data for each group
    race_x = []
    race_y = []
    sex_x = []
    sex_y = []
    other_x = []
    other_y = []
    other_labels = []


    
    if unfairness_distribution:
        race_states = unfairness_distribution.get("race_state", [])
        sex_states = unfairness_distribution.get("sex_states", [])
        for i, client in enumerate(clients):
            if client in race_states:
                race_x.append(fairness2[i])
                race_y.append(fairness1[i])
            elif client in sex_states:
                sex_x.append(fairness2[i])
                sex_y.append(fairness1[i])
    else:
        for i, client in enumerate(clients):
            race_x.append(fairness2[i])
            race_y.append(fairness1[i])
        # other_labels.append(client)


    # Scatter plot for race-related states
    ax.scatter(
        race_x,
        race_y,
        facecolors=race_color,
        edgecolors='black',
        marker='o',
        s=200,
        linewidths=1.2,
        alpha=0.8,
        label='Race-Related States'
    )

    # Scatter plot for sex-related states
    ax.scatter(
        sex_x,
        sex_y,
        facecolors=sex_color,
        edgecolors='black',
        marker='o',
        s=200,
        linewidths=1.2,
        alpha=0.8,
        label='Sex-Related States'
    )

    # Scatter plot for other states (gray)
    # ax.scatter(
    #     other_x,
    #     other_y,
    #     facecolors='gray',
    #     edgecolors='black',
    #     marker='o',
    #     s=200,
    #     linewidths=1.2,
    #     alpha=0.6,
    #     label='Other States'
    # )

    ax.plot(
        [min_val - padding, max_val + padding],
        [min_val - padding, max_val + padding],
        linestyle="dotted",
        color="gray",
        linewidth=2
    )

    ax.tick_params(axis="both", which="major", labelsize=ticks_font_size)
    ax.set_xlim(min_val - padding, max_val + padding)
    ax.set_ylim(min_val - padding, max_val + padding)
    ax.set_xlabel(xlabel, fontsize=label_font_size)
    ax.set_ylabel(ylabel, fontsize=label_font_size)
    ax.set_title(title, fontsize=title_font_size)
    ax.grid(True)

    # Create a separate figure for the legend
    fig_legend = plt.figure(figsize=(6, 1))
    ax_legend = fig_legend.add_subplot(111)

    # Create custom patches for the legend
    race_patch = mpatches.Patch(facecolor=race_color, edgecolor='black', label='Race-Biased State')
    sex_patch = mpatches.Patch(facecolor=sex_color, edgecolor='black', label='Sex-Biased State')

    # Add the legend to the separate axes
    ax_legend.legend(handles=[race_patch, sex_patch], fontsize=24, loc='center', frameon=False, ncol=2)
    ax_legend.axis('off')  # Turn off the axes for the legend

    fig_legend.tight_layout()
    fig_legend.savefig("./images/legend_blue_red.pdf", bbox_inches='tight', dpi=150)

    fig.tight_layout()

    return fig

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



def visualize_value_change(df1: pd.DataFrame, df2: pd.DataFrame, sensitive_col: str = "DP_RACE", value_col: str = "Value", font_size: int = 26, ticks_font_size=20, title:str = "", y_label:str="", initial_state:str=""):
    """
    Visualizes the change in a specified sensitive column across different value
    categories between two dataframes, with refined arrow styling.

    Args:
        df1 (pd.DataFrame): The first dataframe representing the initial state.
        df2 (pd.DataFrame): The second dataframe representing the final state.
        sensitive_col (str, optional): The name of the sensitive column to track.
                                       Defaults to "DP_RACE".
        value_col (str, optional): The name of the column representing the value categories.
                                   Defaults to "Value".
    """
    # Merge the two dataframes based on the common columns
    merged_df = pd.merge(df1, df2, on="dataset", suffixes=('_df1', '_df2'))

    fig, ax = plt.subplots(figsize=(10, 6))

    # Iterate and draw arrows first
    arrow_head_width = 0.05
    arrow_head_length = 0.05
    arrow_alpha = 0.7
    arrow_linewidth = 1.5
    arrow_color = 'gray'

    for index, row in merged_df.iterrows():
        x1 = float(row[f'{value_col}_df1'])
        y1 = float(row[f'{sensitive_col}_df1'])
        x2 = float(row[f'{value_col}_df2'])
        y2 = float(row[f'{sensitive_col}_df2'])

        plt.arrow(x1, y1 , x2 - x1, y2 - y1,
                  head_width=arrow_head_width,
                  head_length=arrow_head_length,
                  fc=arrow_color,
                  ec=arrow_color,
                  alpha=arrow_alpha,
                  linewidth=arrow_linewidth,
                  length_includes_head=True,
                  zorder=1) # Ensure arrows are behind the points

    # Styling for the initial state scatter plot
    marker_face_color = '#004D40' 
    marker_edge_color = 'black' 
    marker_size = 200
    marker_linewidth = 1.2

    # Plot the initial state with the desired style
    for val in df1[value_col].unique():
        y_vals = df1[df1[value_col] == val][sensitive_col]
        x_vals = [int(float(val))] * len(y_vals)
        plt.scatter(
            x_vals,
            y_vals,
            facecolors=marker_face_color,
            edgecolors=marker_edge_color,
            marker='o',
            s=marker_size,
            linewidths=marker_linewidth,
            label=initial_state if val == df1[value_col].unique()[0] else "", # Label only once
            zorder=2, # Ensure initial state points are on top of arrows,
            alpha=0.6
        )

    # Plot the final state points on top of the arrows
    for val in df2[value_col].unique():
        y_vals = df2[df2[value_col] == val][sensitive_col]
        x_vals = [int(float(val))] * len(y_vals)
        plt.scatter(
            x_vals,
            y_vals,
            s=200,
            color='#FFC107',
            edgecolor='black',
            label='FedAVG' if val == df2[value_col].unique()[0] else "", # Label only once
            zorder=2, # Ensure final state points are on top of arrows
            alpha=0.8
        )


    # Customize the x-axis ticks based on the range in df1
    max_val_df1 = int(float(df1[value_col].max()))
    labels = list(range(1, max_val_df1 + 1))
    plt.xticks(ticks=labels, labels=labels, fontsize=ticks_font_size)
    plt.yticks(fontsize=ticks_font_size)
    plt.xlabel('Sensitive Group Value', fontsize=font_size)
    plt.ylabel(y_label, fontsize=font_size)
    plt.title(title, fontsize=font_size)
    plt.grid(True)
    
    # # Move legend outside plot, below
    # ax.legend(loc='upper center',
    #           bbox_to_anchor=(0.5, -0.15),
    #           fontsize=20,
    #           title_fontsize=22,
    #           ncol=2, frameon=False)

    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches

def visualize_value_change(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    sensitive_col: str = "DP_RACE",
    value_col: str = "Value",
    font_size: int = 26,
    ticks_font_size: int = 20,
    
    title: str = "",
    y_label: str = "",
    initial_state: str = "",
    jitter_amount: float = 0.1,
    legend_filename: str = "value_change_legend.pdf",
) -> plt.Figure:
    """
    Visualizes the change in a specified sensitive column across different value
    categories between two dataframes with jittered dots and a separate legend file.
    Arrows are drawn only if the 'Value' is different between the two connected states.

    Args:
        df1 (pd.DataFrame): The first dataframe representing the initial state.
        df2 (pd.DataFrame): The second dataframe representing the final state.
        sensitive_col (str, optional): The name of the sensitive column to track.
                                       Defaults to "DP_RACE".
        value_col (str, optional): The name of the column representing the value categories.
                                   Defaults to "Value".
        font_size (int, optional): Font size for labels and title. Defaults to 26.
        ticks_font_size (int, optional): Font size for axis ticks. Defaults to 20.
        title (str, optional): The title of the plot. Defaults to "".
        y_label (str, optional): The label for the y-axis. Defaults to "".
        initial_state (str, optional): Label for the initial state in the legend. Defaults to "".
        jitter_amount (float, optional): The maximum horizontal shift for the dots.
                                         Defaults to 0.1.
        legend_filename (str, optional): The filename to save the legend.
                                          Defaults to "value_change_legend.png".

    Returns:
        matplotlib.figure.Figure: The matplotlib figure object for the main plot.
    """
    # Merge the two dataframes based on the common columns
    merged_df = pd.merge(df1, df2, on="dataset", suffixes=('_df1', '_df2'))

    fig, ax = plt.subplots(figsize=(12, 5))

    # Styling for the initial state scatter plot
    marker_face_color = '#004D40'
    marker_edge_color = 'black'
    marker_size = 200
    marker_linewidth = 1.2

    # Arrow styling
    arrow_head_width = 0.05
    arrow_head_length = 0.05
    arrow_alpha = 0.7
    arrow_linewidth = 1.5
    arrow_color = 'gray'

    # Store jittered positions
    jittered_positions_df1 = {}
    jittered_positions_df2 = {}

    # Plot the initial state with jitter
    for val in sorted(df1[value_col].unique()):
        subset = df1[df1[value_col] == val]
        y_vals = subset[sensitive_col].astype(float)
        x_base = int(float(val))
        jitter = np.random.uniform(-jitter_amount, jitter_amount, len(y_vals))
        x_vals = [x_base + j for j in jitter]
        jittered_positions_df1[int(float(val))] = list(zip(subset['dataset'].values, x_vals, y_vals.values))
        plt.scatter(
            x_vals,
            y_vals,
            facecolors=marker_face_color,
            edgecolors=marker_edge_color,
            marker='o',
            s=marker_size,
            linewidths=marker_linewidth,
            label=initial_state if val == df1[value_col].unique()[0] else "",  # Label only once
            zorder=2,  # Ensure initial state points are on top of arrows,
            alpha=0.6
        )

    # Plot the final state points with jitter
    for val in sorted(df2[value_col].unique()):
        
        subset = df2[df2[value_col] == val]
        y_vals = subset[sensitive_col].astype(float)
        x_base = int(float(val))
        jitter = np.random.uniform(-jitter_amount, jitter_amount, len(y_vals))
        x_vals = [x_base + j for j in jitter]
        jittered_positions_df2[int(float(val))] = list(zip(subset['dataset'].values, x_vals, y_vals.values))
        plt.scatter(
            x_vals,
            y_vals,
            s=200,
            color='#FFC107',
            edgecolor='black',
            label='FedAVG' if val == df2[value_col].unique()[0] else "",  # Label only once
            zorder=2,  # Ensure final state points are on top of arrows
            alpha=0.8
        )

    # Draw arrows based on the 'dataset' identifier and different 'Value'
    for index, row in merged_df.iterrows():
        dataset_id = row['dataset']
        val1_orig = str(row[f'{value_col}_df1'])
        val2_orig = str(row[f'{value_col}_df2'])
        # if int(val1_orig) != int(val2_orig):
        #     print(val1_orig, val2_orig)
        #     print(2 in jittered_positions_df1)
        #     print(jittered_positions_df2)
        
        if int(float(val1_orig)) in jittered_positions_df1 and int(float(val2_orig)) in jittered_positions_df2 and val1_orig != val2_orig:
            initial_matches = [match for match in jittered_positions_df1[int(float(val1_orig))] if match[0] == dataset_id]
            final_matches = [match for match in jittered_positions_df2[int(float(val2_orig))] if match[0] == dataset_id]

            if initial_matches and final_matches:
                initial_x, initial_y = initial_matches[0][1], initial_matches[0][2]
                final_x, final_y = final_matches[0][1], final_matches[0][2]
                plt.arrow(initial_x, initial_y, final_x - initial_x, final_y - initial_y,
                          head_width=arrow_head_width,
                          head_length=arrow_head_length,
                          fc=arrow_color,
                          ec=arrow_color,
                          alpha=arrow_alpha,
                          linewidth=arrow_linewidth,
                          length_includes_head=True,
                          zorder=1)

                print("Arrow")

    # Customize the x-axis ticks based on the range in df1
    max_val_df1 = int(float(df1[value_col].max()))
    labels = list(range(1, max_val_df1 + 1))
    plt.xticks(ticks=labels, labels=labels, fontsize=ticks_font_size)
    plt.yticks(fontsize=ticks_font_size)
    plt.xlabel('Sensitive Group Value', fontsize=font_size)
    plt.ylabel(y_label, fontsize=font_size)
    plt.title(title, fontsize=font_size)
    plt.grid(True)

    # Create a separate figure for the legend
    fig_legend = plt.figure(figsize=(6, 1))  # Adjust figure size as needed
    ax_legend = fig_legend.add_subplot(111)

    # Create custom patches for the legend
    initial_patch = mpatches.Patch(facecolor='#004D40', edgecolor='black', alpha=0.6, label=initial_state)
    fedavg_patch = mpatches.Patch(facecolor='#FFC107', edgecolor='black', alpha=0.8, label='FedAVG')

    # Add the legend to the separate axes
    ax_legend.legend(handles=[initial_patch, fedavg_patch], fontsize=20, loc='center', frameon=False, ncol=2)
    ax_legend.axis('off')  # Turn off the axes for the legend

    fig_legend.tight_layout()
    fig_legend.savefig(legend_filename)

    plt.tight_layout()
    return fig

# Cross Silo

## VALUE Experiment

In [ ]:
folder_name = "baseline_value_cross_silo"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_silo_value_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/hhqyjzsl")
df = pd.DataFrame(run.scan_history())

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}


In [ ]:
results = {}

for node in range(0, 51):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)].split("_")[0]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
results_df["Value"] = [int(i) for i in results_df["Value"]]
results_df.head()

In [ ]:
# dp_sex_centralized = dp[dp["model"] == "LogisticRegression"][["DP_RACE", "dataset"]]
# results_fedavg = results_df[["DP_RACE", "dataset"]]

# # Merge the two DataFrames on the "dataset" column
# merged_df = pd.merge(dp_sex_centralized, results_fedavg, on="dataset", suffixes=('_centralized', '_federated'))
# merged_df.head(100)

In [ ]:
# Before and after for cross-silo experiment with Value experiment
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model
    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=18,
    )
    plot.savefig(f"./images/before_after_cross_silo_value_race_{model}.pdf")

# Before and after for cross-silo experiment with Value experiment
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model
    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
    )
    plot.savefig(f"./images/before_after_cross_silo_value_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
# plot_comparison_fairness(df=results_df, title="FedAvg")

In [ ]:
# plot_comparison_fairness(df=dp.loc[dp["model"] == "LogisticRegression"], title="LogisticRegression")

In [ ]:
logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in logistic_regression.iterrows()]

df_sex_unfairness_logistic = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in logistic_regression.iterrows()]

df_race_unfairness_logistic = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


xgboost = dp[dp["model"] == "XGBoost"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in xgboost.iterrows()]

df_sex_unfairness_xgboost = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

xgboost = dp[dp["model"] == "XGBoost"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in xgboost.iterrows()]

df_race_unfairness_xgboost = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


In [ ]:


create_value_plot(df_sex_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_sex_logistic_dem_parity_value", save=True)
create_value_plot(df_race_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_race_logistic_dem_parity_value", save=True)

create_value_plot(df_sex_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_sex_xgboost_dem_parity_value", save=True)
create_value_plot(df_race_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_race_xgboost_dem_parity_value", save=True)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_logistic_cross_silo_value")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_xgboost_cross_silo_value")

In [ ]:
df_race_unfairness_xgboost["Value"] = [int(float(i)) for i in df_race_unfairness_xgboost["Value"]]
fig = visualize_value_change(df1=df_race_unfairness_xgboost, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="XGBoost", jitter_amount=0.1)

fig.savefig(f"./images/arrow_silo_value_xgboost.pdf", bbox_inches='tight', dpi=150)

In [ ]:
df_race_unfairness_logistic["Value"] = [int(float(i)) for i in df_race_unfairness_logistic["Value"]]
fig = visualize_value_change(df1=df_race_unfairness_logistic, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="Log. Regression")

fig.savefig(f"./images/arrow_silo_value_lr.pdf", bbox_inches='tight', dpi=150)

In [ ]:
values_1 = {row["dataset"]:row["Value"] for index, row in df_race_unfairness_logistic.iterrows()}
values_2 = {row["dataset"]:row["Value"] for index, row in results_df.iterrows()}

In [ ]:
values_1 = {row["dataset"]:row["Value"] for index, row in df_race_unfairness_logistic.iterrows()}
values_2 = {row["dataset"]:row["Value"] for index, row in results_df.iterrows()}
counter = 0
for key in values_1:
    if values_1[key] != values_2[key]:
        counter+=1
print(counter)

## Attribute Experiment

In [ ]:
folder_name = "baseline_attribute_cross_silo"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_silo_attribute_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/reb9abt2")
df = pd.DataFrame(run.scan_history())

states_unfairness = {
    "sex_states":['SD', 'IN', 'WV', 'PA', 'IL', 'MI', 'WA', 'TX', 'MO', 'WY', 'TN', 'OK', 'UT', 'ID', 'ND', 'VA', 'AR', 'KS', 'NH', 'OH', 'LA'],
    "race_state":['AL', 'NM', 'IA', 'MA', 'FL', 'AZ', 'NY', 'AK', 'MS', 'NC', 'GA', 'VT', 'SC', 'NJ', 'CT', 'DE', 'RI', 'WI', 'OR', 'NV', 'NE', 'MN', 'CA', 'MT', 'MD', 'CO', 'HI', 'KY', 'ME']
}

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
results = {}

for node in range(0, 51):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)].split("_")[0]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
results_df.head()

In [ ]:
# dp_sex_centralized = dp[dp["model"] == "LogisticRegression"][["DP_RACE", "dataset"]]
# results_fedavg = results_df[["DP_RACE", "dataset"]]

# # Merge the two DataFrames on the "dataset" column
# merged_df = pd.merge(dp_sex_centralized, results_fedavg, on="dataset", suffixes=('_centralized', '_federated'))
# merged_df.head(100)

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Parity",
        xlabel="FedAVG Dem. Parity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/before_after_cross_silo_attribute_race_{model}.pdf")

for model in dp["model"].unique():
    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Parity",
        xlabel="FedAVG Dem. Parity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/before_after_cross_silo_attribute_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
# DP Sex on the y and DP Race on the X
for model in dp["model"].unique():
    fig = scatter_fairness_plot(
        df1=dp.loc[dp["model"] == model],
        client_column="dataset",
        fairness_column_X="DP_RACE",
        fairness_column_Y="DP_SEX",
        ylabel=f"Dem. Disparity SEX",
        xlabel="Dem. Disparity RACE",
        title=f"Attribute Bias Distribution",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )

    fig.savefig(f"./images/{model}_attribute.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity")
diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity")

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_logistic_cross_silo_attribute")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_xgboost_cross_silo_attribute")

# Cross-Device

## Attribute

In [ ]:
folder_name = "baseline_attribute_cross_device"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_device_attribute_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/7xd7s7vf")
df = pd.DataFrame(run.scan_history())
states_unfairness = {
    "sex_states":['IN_0', 'IN_2', 'IN_3', 'WV_0', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_5', 'IL_2', 'IL_3', 'IL_4', 'MI_0', 'MI_1', 'MI_2', 'MI_3', 'MI_4', 'WA_3', 'WA_5', 'TX_0', 'TX_1', 'TX_3', 'TX_4', 'TX_5', 'MO_1', 'MO_4', 'TN_2', 'TN_4', 'TN_5', 'OK_0', 'OK_2', 'OK_3', 'NE_5', 'UT_0', 'UT_4', 'UT_5', 'ID_0', 'ND_1', 'ND_4', 'VA_0', 'VA_1', 'VA_3', 'VA_4', 'KS_2', 'KS_4', 'NH_0', 'NH_1', 'NH_4', 'NH_5', 'OH_0', 'OH_1', 'OH_2', 'OH_3', 'OH_4', 'LA_4'],
    "race_state":['AL_2', 'AL_3', 'AL_4', 'NM_1', 'NM_5', 'SD_3', 'IA_0', 'IA_1', 'MA_0', 'MA_3', 'MA_4', 'WV_5', 'FL_5', 'AZ_0', 'AZ_5', 'NY_0', 'NY_1', 'NY_2', 'NY_3', 'NY_4', 'NY_5', 'MS_4', 'MS_5', 'NC_1', 'SC_0', 'SC_1', 'SC_2', 'SC_4', 'NJ_1', 'NJ_3', 'NJ_4', 'CT_2', 'CT_5', 'RI_2', 'WI_2', 'WI_3', 'OR_1', 'OR_3', 'OR_4', 'NV_0', 'NV_1', 'NV_3', 'NE_3', 'NE_4', 'UT_1', 'MN_0', 'MN_3', 'MN_4', 'MN_5', 'ID_2', 'CA_1', 'CO_0', 'CO_3', 'LA_2', 'HI_0', 'ME_1'],
}

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
results = {}

for node in range(0, 111):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
# dp_sex_centralized = dp[dp["model"] == "LogisticRegression"][["DP_RACE", "dataset"]]
# results_fedavg = results_df[["DP_RACE", "dataset"]]

# # Merge the two DataFrames on the "dataset" column
# merged_df = pd.merge(dp_sex_centralized, results_fedavg, on="dataset", suffixes=('_centralized', '_federated'))
# merged_df.head(100)

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model
    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model],
        df2=results_df,
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/before_after_cross_device_attribute_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model
    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/before_after_cross_device_attribute_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
# DP Sex on the y and DP Race on the X
for model in dp["model"].unique():
    fig = scatter_fairness_plot(
        df1=dp.loc[dp["model"] == model],
        client_column="dataset",
        fairness_column_X="DP_RACE",
        fairness_column_Y="DP_SEX",
        ylabel=f"Dem. Disparity SEX",
        xlabel="Dem. Disparity RACE",
        title=f"Attribute Bias Distribution",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )

    fig.savefig(f"./images/{model}_attribute_cross_device.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_logistic_cross_device_attribute")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="bar_plot_differences_xgboost_cross_device_attribute")

## Value

In [ ]:
folder_name = "baseline_value_cross_device"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_device_value_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/dyefdq2e")
df = pd.DataFrame(run.scan_history())

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
results = {}

for node in range(0, 100):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
results_df.head()

In [ ]:
# dp_sex_centralized = dp[dp["model"] == "LogisticRegression"][["DP_RACE", "dataset"]]
# results_fedavg = results_df[["DP_RACE", "dataset"]]

# # Merge the two DataFrames on the "dataset" column
# merged_df = pd.merge(dp_sex_centralized, results_fedavg, on="dataset", suffixes=('_centralized', '_federated'))
# merged_df.head(100)

In [ ]:
# Before and after for cross-silo experiment with Value experiment
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
    )
    plot.savefig(f"./images/before_after_cross_device_value_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
    )
    plot.savefig(f"./images/before_after_cross_device_value_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
# plot_comparison_fairness(df=results_df, title="FedAvg")

In [ ]:
# plot_comparison_fairness(df=dp.loc[dp["model"] == "LogisticRegression"], title="LogisticRegression")

In [ ]:
logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in logistic_regression.iterrows()]

df_sex_unfairness_logistic = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in logistic_regression.iterrows()]

df_race_unfairness_logistic = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


xgboost = dp[dp["model"] == "XGBoost"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in xgboost.iterrows()]

df_sex_unfairness_xgboost = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

xgboost = dp[dp["model"] == "XGBoost"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in xgboost.iterrows()]

df_race_unfairness_xgboost = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


In [ ]:


create_value_plot(df_sex_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_sex_logistic_dem_parity_value_cross_device", save=True)
create_value_plot(df_race_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_race_logistic_dem_parity_value_cross_device", save=True)

create_value_plot(df_sex_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_sex_xgboost_dem_parity_value_cross_device", save=True)
create_value_plot(df_race_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="dp_race_xgboost_dem_parity_value_cross_device", save=True)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Demographic Parity", save=True, fig_name="bar_plot_differences_logistic_cross_device_value")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Demographic Parity", save=True, fig_name="bar_plot_differences_xgboost_cross_device_value")

In [ ]:
states = results_df["dataset"]

In [ ]:
filtered = df_race_unfairness_xgboost[df_race_unfairness_xgboost["dataset"].isin(states)]

In [ ]:
filtered

In [ ]:
df_race_unfairness_xgboost["Value"] = [int(float(i)) for i in df_race_unfairness_xgboost["Value"]]
fig = visualize_value_change(df1=filtered, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="XGBoost")

fig.savefig(f"./images/arrow_device_value_xgboost.pdf", bbox_inches='tight', dpi=150)

In [ ]:
filtered = df_race_unfairness_logistic[df_race_unfairness_logistic["dataset"].isin(states)]
print(filtered)
values_1 = {row["dataset"]:row["Value"] for index, row in filtered.iterrows()}
values_2 = {row["dataset"]:row["Value"] for index, row in results_df.iterrows()}
counter = 0
for key in values_1:
    if key in values_2 and values_1[key] != values_2[key]:
        counter+=1
print(counter)
df_race_unfairness_logistic["Value"] = [int(float(i)) for i in df_race_unfairness_logistic["Value"]]
fig = visualize_value_change(df1=filtered, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="Log. Regression")

fig.savefig(f"./images/arrow_device_value_lr.pdf", bbox_inches='tight', dpi=150)

# Unfairness Reduction

## Cross-Silo Attribute

In [ ]:
folder_name = "baseline_attribute_cross_silo"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_silo_attribute_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/zw4xdavp")
df = pd.DataFrame(run.scan_history())



attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
states_unfairness = {
    "sex_states": ['SD', 'IN', 'WV', 'PA', 'IL', 'MI', 'WA', 'TX', 'MO', 'WY', 'TN', 'OK', 'UT', 'ID', 'ND', 'VA', 'AR', 'KS', 'NH', 'OH', 'LA'],
    "race_state": ['AL', 'NM', 'IA', 'MA', 'FL', 'AZ', 'NY', 'AK', 'MS', 'NC', 'GA', 'VT', 'SC', 'NJ', 'CT', 'DE', 'RI', 'WI', 'OR', 'NV', 'NE', 'MN', 'CA', 'MT', 'MD', 'CO', 'HI', 'KY', 'ME']
}

In [ ]:
results = {}

for node in range(0, 51):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)].split("_")[0]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
# plot_comparison_fairness(df=results_df, title="Unfairness reduction")

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model],
        df2=results_df,
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/puffle_before_after_cross_silo_attribute_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/puffle_before_after_cross_silo_attribute_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_logistic_cross_silo_attribute_puffle")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_xgboost_cross_silo_attribute_puffle")

## Cross-Silo Value

In [ ]:
folder_name = "baseline_value_cross_silo"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_silo_value_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/g0bvrdjf")
df = pd.DataFrame(run.scan_history())



attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
results = {}

for node in range(0, 51):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)].split("_")[0]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
# plot_comparison_fairness(df=results_df, title="Unfairness reduction")

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model],
        df2=results_df,
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/puffle_before_after_silo_value_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/puffle_before_after_silo_value_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_logistic_cross_silo_value_puffle")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_xgboost_cross_silo_value_puffle")

In [ ]:
logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in logistic_regression.iterrows()]

df_sex_unfairness_logistic = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

logistic_regression = dp[dp["model"] == "LogisticRegression"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in logistic_regression.iterrows()]

df_race_unfairness_logistic = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


xgboost = dp[dp["model"] == "XGBoost"]
dp_sex_unfairness = [(row["dataset"], row["value_DP_SEX"].split("_")[-1], float(row["DP_SEX"])) for _, row in xgboost.iterrows()]

df_sex_unfairness_xgboost = pd.DataFrame(dp_sex_unfairness, columns =['dataset', 'Value', 'DP_SEX'])

xgboost = dp[dp["model"] == "XGBoost"]
dp_race_unfairness = [(row["dataset"], row["value_DP_RACE"].split("_")[-1], float(row["DP_RACE"])) for _, row in xgboost.iterrows()]

df_race_unfairness_xgboost = pd.DataFrame(dp_race_unfairness, columns =['dataset', 'Value', 'DP_RACE'])


In [ ]:
create_value_plot(df_sex_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="puffle_dp_sex_logistic_dem_parity_value_cross_silo", save=True)
create_value_plot(df_race_unfairness_logistic, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="puffle_dp_race_logistic_dem_parity_value_cross_silo", save=True)

create_value_plot(df_sex_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_SEX", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="puffle_dp_sex_xgboost_dem_parity_value_cross_silo", save=True)
create_value_plot(df_race_unfairness_xgboost, y_label="Dem. Disparity", title="Value Bias Distribution", attribute="DP_RACE", font_size_labels=26, font_size_title=26, font_size_ticks=20, file_name="puffle_dp_race_xgboost_dem_parity_value_cross_silo", save=True)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Demographic Parity", save=True, fig_name="puffle_bar_plot_differences_logistic_cross_silo_value")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Demographic Parity", save=True, fig_name="puffle_bar_plot_differences_xgboost_cross_silo_value")

In [ ]:
states = results_df["dataset"]

filtered = df_race_unfairness_logistic[df_race_unfairness_logistic["dataset"].isin(states)]
print(filtered)
values_1 = {row["dataset"]:row["Value"] for index, row in filtered.iterrows()}
values_2 = {row["dataset"]:row["Value"] for index, row in results_df.iterrows()}
counter = 0
for key in values_1:
    if key in values_2 and values_1[key] != values_2[key]:
        counter+=1
print(counter)
df_race_unfairness_logistic["Value"] = [int(float(i)) for i in df_race_unfairness_logistic["Value"]]
fig = visualize_value_change(df1=filtered, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="Log. Regression")

fig.savefig(f"./images/puffle_arrow_silo_value_lr.pdf", bbox_inches='tight', dpi=150)

In [ ]:
states = results_df["dataset"]

filtered = df_race_unfairness_xgboost[df_race_unfairness_xgboost["dataset"].isin(states)]
print(filtered)
values_1 = {row["dataset"]:row["Value"] for index, row in filtered.iterrows()}
values_2 = {row["dataset"]:row["Value"] for index, row in results_df.iterrows()}
counter = 0
for key in values_1:
    if key in values_2 and values_1[key] != values_2[key]:
        counter+=1
print(counter)
df_race_unfairness_xgboost["Value"] = [int(float(i)) for i in df_race_unfairness_xgboost["Value"]]
fig = visualize_value_change(df1=filtered, df2=results_df, title="Change in Max. Value Disparity", y_label="Dem.Disparity", font_size=26, ticks_font_size=20, initial_state="XGBoost")

fig.savefig(f"./images/puffle_arrow_silo_value_xgboost.pdf", bbox_inches='tight', dpi=150)

## Cross-Device Attribute

In [ ]:
folder_name = "baseline_attribute_cross_device"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_device_attribute_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/qu2rhc1e")
df = pd.DataFrame(run.scan_history())

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}
states_unfairness = {
    "sex_states": ['IN_0', 'IN_2', 'IN_3', 'WV_0', 'PA_0', 'PA_1', 'PA_2', 'PA_3', 'PA_5', 'IL_2', 'IL_3', 'IL_4', 'MI_0', 'MI_1', 'MI_2', 'MI_3', 'MI_4', 'WA_3', 'WA_5', 'TX_0', 'TX_1', 'TX_3', 'TX_4', 'TX_5', 'MO_1', 'MO_4', 'TN_2', 'TN_4', 'TN_5', 'OK_0', 'OK_2', 'OK_3', 'NE_5', 'UT_0', 'UT_4', 'UT_5', 'ID_0', 'ND_1', 'ND_4', 'VA_0', 'VA_1', 'VA_3', 'VA_4', 'KS_2', 'KS_4', 'NH_0', 'NH_1', 'NH_4', 'NH_5', 'OH_0', 'OH_1', 'OH_2', 'OH_3', 'OH_4', 'LA_4'],
    "race_state": ['AL_2', 'AL_3', 'AL_4', 'NM_1', 'NM_5', 'SD_3', 'IA_0', 'IA_1', 'MA_0', 'MA_3', 'MA_4', 'WV_5', 'FL_5', 'AZ_0', 'AZ_5', 'NY_0', 'NY_1', 'NY_2', 'NY_3', 'NY_4', 'NY_5', 'MS_4', 'MS_5', 'NC_1', 'SC_0', 'SC_1', 'SC_2', 'SC_4', 'NJ_1', 'NJ_3', 'NJ_4', 'CT_2', 'CT_5', 'RI_2', 'WI_2', 'WI_3', 'OR_1', 'OR_3', 'OR_4', 'NV_0', 'NV_1', 'NV_3', 'NE_3', 'NE_4', 'UT_1', 'MN_0', 'MN_3', 'MN_4', 'MN_5', 'ID_2', 'CA_1', 'CO_0', 'CO_3', 'LA_2', 'HI_0', 'ME_1']
}

In [ ]:
results = {}

for node in range(0, 111):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model],
        df2=results_df,
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,
    )
    plot.savefig(f"./images/puffle_before_after_cross_device_attribute_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
        unfairness_distribution=states_unfairness,

    )
    plot.savefig(f"./images/puffle_before_after_cross_device_attribute_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_logistic_cross_device_attribute_puffle")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_xgboost_cross_device_attribute_puffle")

## Cross-Device Value

In [ ]:
dddd

In [ ]:
folder_name = "baseline_value_cross_device"
dp = pd.read_csv(f'./{folder_name}/model_perf_DP.csv')
eo = pd.read_csv(f'./{folder_name}/model_perf_EO.csv')
with open ('../data/cross_device_value_final/FL_data/federated/partitions_names.json') as f:
    partition_names = json.load(f)
api = wandb.Api()
run = api.run("/lucacorbucci/Fair_Fed_Dataset_results/runs/0rmxjt18")
df = pd.DataFrame(run.scan_history())

attributes = ["- First DP NEW.", "- Second DP NEW.", "- Third DP NEW.", "- Acc.", "- Group 3"]
mapping = {
    "- Third DP NEW.": "DP_RACE",
    "- Second DP NEW.": "DP_MAR",
    "- First DP NEW.": "DP_SEX",
    "- Acc.": "accuracy",
    "- Group 3": "Value"
}

In [ ]:
results = {}

for node in range(0, 200):
    for attribute in attributes:
        current_attribute = f"{attribute_name} {node} {attribute}"
        if current_attribute in df.columns:
            # Get the values for the current attribute
            values = df[current_attribute].values
            # remove the NaN values
            values = values[~pd.isna(values)]
            # get the last value
            last_value = values[-1]
            if node not in results:
                results[node] = {}
            results[node][mapping[attribute]] = last_value
            results[node]["dataset"] = partition_names[str(node)]

results_df = pd.DataFrame.from_dict(results, orient='index')

In [ ]:
# plot_comparison_fairness(df=results_df, title="Unfairness reduction")

In [ ]:
# Before and after for the attribute experiment with Cross-silo
for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model],
        df2=results_df,
        client_column="dataset",
        fairness_column="DP_RACE",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="RACE",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,
    )
    plot.savefig(f"./images/puffle_before_after_cross_device_attribute_race_{model}.pdf")

for model in dp["model"].unique():
    model_title = "Log. Regr." if model == "LogisticRegression" else model

    plot = local_client_fairness_plot(
        df1=dp[dp["model"] == model], 
        df2=results_df, 
        client_column="dataset",
        fairness_column="DP_SEX",
        ylabel=f"{model_title} Dem. Disparity",
        xlabel="FedAVG Dem. Disparity",
        title="SEX",
        title_font_size=26, 
        label_font_size=26,
        ticks_font_size=20,

    )
    plot.savefig(f"./images/puffle_before_after_cross_device_attribute_sex_{model}.pdf", bbox_inches='tight', dpi=150)

In [ ]:
diff_df = compute_differences(dp.loc[dp["model"] == "LogisticRegression"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="Logistic Regression - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_logistic_cross_silo_attribute_puffle")

diff_df = compute_differences(dp.loc[dp["model"] == "XGBoost"], results_df)
diff_df = diff_df.dropna(subset=["DP_SEX"])
bar_plot_differences(diff_df, list(diff_df["dataset"]), title="XGBoost - FedAvg (Puffle)", font_size_title=26, font_size_ticks=20, font_size_labels=26, y_axis="Dem. Disparity", save=True, fig_name="puffle_bar_plot_differences_xgboost_cross_silo_attribute_puffle")

# Dataset Disparity

In [ ]:
# def pre_process_income(df):
#     """
#     Pre-process the income dataset to make it ready for the simulation
#     In this function we consider "SEX" as the sensitive value and "PINCP" as the target value.

#     Args:
#         data: the raw data
#         years_list: the list of years to be considered
#         states_list: the list of states to be considered

#     Returns:
#         Returns a list of pre-processed data for each state, if multiple years are
#         selected, the data are concatenated.
#         We return three lists:
#         - The first list contains a pandas dataframe of features for each state
#         - The second list contains a pandas dataframe of labels for each state
#         - The third list contains a pandas dataframe of groups for each state
#         The values in the list are numpy array of the dataframes
#     """

#     categorical_columns = ["COW", "SCHL"] #, "RAC1P"]
#     continuous_columns = ["AGEP", "WKHP", "OCCP", "POBP", "RELP"]

#     # get the target and sensitive attributes
#     target_attributes = df[">50K"]
#     sensitive_attributes = df["SEX"]

#     # convert the columns to one-hot encoding
#     df = pd.get_dummies(df, columns=categorical_columns, dtype=int)

#     # normalize the continuous columns between 0 and 1
#     for col in continuous_columns:
#         df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

#     return pd.DataFrame(df)


# def pre_process_single_datasets(df):
#     dataframe = pd.DataFrame()
#     label = pd.DataFrame()
#     group = pd.DataFrame()
#     second_group = pd.DataFrame()
#     third_group = pd.DataFrame()
#     dataframes = []
#     labels = []
#     groups = []
#     second_groups = []
#     third_groups = []
#     target_attributes = df[">50K"]
#     sensitive_attributes = df["SEX"]
#     second_sensitive_attributes = df["MAR"]
    
#     third_sensitive_attributes = df["RAC1P"]
#     third_sensitive_attributes = third_sensitive_attributes.astype(int)
#     target_attributes = target_attributes.astype(int)

#     sensitive_attributes = [1 if item == 1 else 0 for item in sensitive_attributes]

#     second_sensitive_attributes = [
#         1 if item == 1 else 0 for item in second_sensitive_attributes
#     ]

#     third_sensitive_attributes = [
#         1 if item == 1 else 0 for item in third_sensitive_attributes
#     ]

#     df = df.drop([">50K"], axis=1)
#     # df.drop(['RAC1P_1.0', 'RAC1P_2.0'], axis=1, inplace=True)

#     # concatenate the dataframes
#     dataframe = pd.concat([dataframe, df])
#     # remove RAC1P from dataframe

#     # convert the labels and groups to dataframes
#     label = pd.concat([label, pd.DataFrame(target_attributes)])
#     group = pd.concat([group, pd.DataFrame(sensitive_attributes)])
#     second_group = pd.concat([second_group, pd.DataFrame(second_sensitive_attributes)])
#     third_group = pd.concat([third_group, pd.DataFrame(third_sensitive_attributes)])

#     assert len(dataframe) == len(label) == len(group) == len(second_group)
#     dataframes.append(dataframe.to_numpy())
#     labels.append(label.to_numpy())
#     groups.append(group.to_numpy())
#     second_groups.append(second_group.to_numpy())
#     third_groups.append(third_group.to_numpy())
#     return dataframes, labels, groups, second_groups, third_groups

In [ ]:
# from sklearn.model_selection import train_test_split

# # open the original csv files: 
# folder = "../original_data/cross_silo_value_final/"
# list_files = !ls {folder}
# unfair_dfs = []
# print(list_files)

# states = ['CT',
#  'RI',
#  'VT',
#  'TX',
#  'GA',
#  'PR',
#  'OH',
#  'NE',
#  'HI',
#  'MO',
#  'PA',
#  'DE',
#  'WV',
#  'MD',
#  'AZ',
#  'LA',
#  'WA',
#  'TN',
#  'MA',
#  'NJ',
#  'ME',
#  'SC',
#  'MI',
#  'OK',
#  'IL',
#  'FL',
#  'UT',
#  'AK',
#  'WI',
#  'NH',
#  'VA',
#  'SD',
#  'MS',
#  'ND',
#  'NC',
#  'AL',
#  'IA',
#  'ID',
#  'WY',
#  'NV',
#  'NM',
#  'NY',
#  'CA',
#  'AR',
#  'MN',
#  'OR',
#  'MT',
#  'KY',
#  'KS',
#  'IN',
#  'CO']

# partitions_names = []

# for state in states:
#     partitions = set()
#     for file in list_files:
#         if file.endswith(".csv"):
#             partition = int(file.split("_")[-1].split(".")[0])
#             if partition not in partitions:
#                 partitions_names.append(f"{state}_{partition}")
#                 partitions.add(partition)
#                 train = pd.read_csv(f"{folder}{state}_{partition}.csv")
#                 # split the train csv into train and test
#                 train, test = train_test_split(train, test_size=0.2)

#                 unfair_dfs.append(train)
#                 unfair_dfs.append(test)

In [ ]:
# concatenated_df = pd.concat(unfair_dfs, ignore_index=True)
# concatenated_df["PINCP"] = [1 if item == True else 0 for item in concatenated_df["PINCP"]]

# # rename the column PINCP to >50K
# concatenated_df.rename(columns={"PINCP": ">50K"}, inplace=True)
# concatenated_df.drop(["__index_level_0__"], axis=1, inplace=True)

# concatenated_df.head()

In [ ]:
# # Apply one-hot encoding
# pre_processed_df = pre_process_income(concatenated_df)

# split_dfs = []
# start_idx = 0
# for df in unfair_dfs:
#     end_idx = start_idx + len(df)
#     split_dfs.append(pre_processed_df.iloc[start_idx:end_idx])
#     start_idx = end_idx

In [ ]:
# def demographic_parity_difference(y_pred, sensitive_features):
#     """
#     Computes the demographic parity difference between two numpy arrays.

#     This function calculates the demographic parity difference, which is the
#     largest absolute difference in the acceptance rates between any two groups
#     defined by the sensitive features.

#     Args:
#         y_pred : array-like of shape (n_samples,)
#             Predicted labels.
#         sensitive_features : array-like of shape (n_samples,)
#             Sensitive features.

#     Returns:
#         float
#             The demographic parity difference.
#     """
#     # Ensure input arrays are numpy arrays
#     y_pred = np.asarray(y_pred)
#     sensitive_features = np.asarray(sensitive_features)

#     # Check for consistent lengths
#     if not (y_pred.shape == sensitive_features.shape):
#         raise ValueError("y_pred and sensitive_features must have the same shape")

#     # Get unique values in the sensitive features array
#     unique_sensitive_values = np.unique(sensitive_features)

#     # Calculate acceptance rates for each group
#     acceptance_rates = []
#     for group_value in unique_sensitive_values:
#         group_indices = (sensitive_features == group_value)
#         acceptance_rate = np.mean(y_pred[group_indices])
#         acceptance_rates.append(acceptance_rate)

#     # Calculate the demographic parity difference
#     max_diff = 0
#     for i in range(len(acceptance_rates)):
#         for j in range(i + 1, len(acceptance_rates)):
#             diff = np.abs(acceptance_rates[i] - acceptance_rates[j])
#             if diff > max_diff:
#                 max_diff = diff

#     return max_diff


In [ ]:
# dataset_analysis = {}

# counter = {
#     "sex": 0,
#     "race": 0, 
#     "mar": 0,
# }

# for index in range(0, len(split_dfs), 2):
#     train_state = split_dfs[index]
#     test_state = split_dfs[index + 1]
#     print(len(train_state), len(test_state))
#     (
#         train_data,
#         train_labels,
#         train_groups,
#         train_second_groups,
#         train_third_groups,
#     ) = pre_process_single_datasets(train_state)
#     (
#         test_data,
#         test_labels,
#         test_groups,
#         test_second_groups,
#         test_third_groups,
#     ) = pre_process_single_datasets(test_state)


#     SEX = test_groups[0]
#     # MAR = test_second_groups[0]
#     RACE = test_third_groups[0]
#     labels = test_labels[0]

#     dp_sex = demographic_parity_difference(SEX, labels)
#     # dp_mar = demographic_parity_difference(MAR, labels)
#     dp_race = demographic_parity_difference(RACE, labels)


#     if dp_sex > dp_race:
#         max_disparity = "sex" 
#     else: 
#         max_disparity = "race"
    
#     counter[max_disparity] += 1

#     node = index // 2
#     if node not in dataset_analysis:
#         dataset_analysis[node] = {}
#     dataset_analysis[node][mapping[attribute]] = last_value
#     dataset_analysis[node]["dataset"] = partition_names[str(node)].split("_")[0]
